In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm
import gradio as gr
import os
import google.generativeai as genai

In [2]:
vocab_size = 32000
embedding_dim = 256
batch_size = 32 
learning_rate = 0.001 
heads = 4
epochs = 400
seq_len = 300
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
sp = spm.SentencePieceProcessor()
sp.load("poetry_model.model")

True

In [4]:
class WordEmbedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    def forward(self, x):
        return self.embeddings(x)

In [5]:
def positional_encoding(seq_len, embedding_dim, device=device):
        positional_encoding = torch.zeros(seq_len, embedding_dim)
        position = torch.arange(0, seq_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / embedding_dim))
        positional_encoding[:, 0::2] = torch.sin(position * div_term)
        positional_encoding[:, 1::2] = torch.cos(position * div_term)
        return positional_encoding

In [6]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super(MultiHeadSelfAttention, self).__init__()
        self.heads = heads

        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        self.softmax = nn.Softmax(dim=-1)

        self.w_a = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)) / torch.sqrt(torch.tensor(embedding_dim))), V).to(device)

        attention = attention.transpose(1, 2).reshape(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [7]:
class MaskedMultiHeadSelfAttention(nn.Module):
    def __init__(self, heads, embedding_dim):
        super(MaskedMultiHeadSelfAttention, self).__init__()
        self.heads = heads

        self.w_q = nn.Linear(embedding_dim, embedding_dim)
        self.w_k = nn.Linear(embedding_dim, embedding_dim)
        self.w_v = nn.Linear(embedding_dim, embedding_dim)

        self.softmax = nn.Softmax(dim=-1)

        self.w_a = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, embedding_vector):
        batch_size, seq_len, embedding_dim = embedding_vector.size()
        Q = self.w_q(embedding_vector)
        K = self.w_k(embedding_vector)
        V = self.w_v(embedding_vector)

        head_dim = embedding_dim // self.heads
        Q = Q.view(batch_size, self.heads, seq_len, head_dim)
        K = K.view(batch_size, self.heads, seq_len, head_dim)
        V = V.view(batch_size, self.heads, seq_len, head_dim)

        mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool), diagonal=1).unsqueeze(0).unsqueeze(0).expand(batch_size, self.heads, -1, -1).to(device)
        value_to_fill = float('-inf')

        attention = torch.matmul(self.softmax(torch.matmul(Q, K.transpose(-1, -2)).masked_fill(mask, value_to_fill) / torch.sqrt(torch.tensor(embedding_dim))), V)
        attention = attention.transpose(1, 2).reshape(batch_size, seq_len, embedding_dim)
        output = self.w_a(attention)
        return output

In [8]:
class AddNorm(nn.Module):
    def __init__(self, n_features):
        super(AddNorm, self).__init__()
        self.norm = nn.LayerNorm(n_features)

    def forward(self, original, modified):
        return self.norm(original + modified)

In [9]:
class FeedForward(nn.Module):
    def __init__(self, embedding_dim):
        super(FeedForward, self).__init__()
        self.lr1 = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return x

In [10]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, batch_size, seq_len, heads):
        super(Decoder, self).__init__()
        self.word_embedder = WordEmbedder(vocab_size, embedding_dim)
        self.masked_attention = MaskedMultiHeadSelfAttention(heads, embedding_dim)
        self.add_norm1 = AddNorm(embedding_dim)
        self.attention = MultiHeadSelfAttention(heads, embedding_dim)
        self.add_norm2 = AddNorm(embedding_dim)
        self.feed_forward = FeedForward(embedding_dim)
        self.add_norm3 = AddNorm(embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.word_embedder(x)
        x += positional_encoding(x.shape[1], x.shape[2]).unsqueeze(0).expand(x.shape[0], x.shape[1], x.shape[2]).to(device)
        x = self.add_norm1(x, self.masked_attention(x))
        x = self.add_norm2(x, self.attention(x))
        x = self.add_norm3(x, self.feed_forward(x))
        logits = self.linear(x)
        return logits

In [11]:
model = Decoder(vocab_size, embedding_dim, batch_size, seq_len, heads).to(device)
model

Decoder(
  (word_embedder): WordEmbedder(
    (embeddings): Embedding(32000, 256)
  )
  (masked_attention): MaskedMultiHeadSelfAttention(
    (w_q): Linear(in_features=256, out_features=256, bias=True)
    (w_k): Linear(in_features=256, out_features=256, bias=True)
    (w_v): Linear(in_features=256, out_features=256, bias=True)
    (softmax): Softmax(dim=-1)
    (w_a): Linear(in_features=256, out_features=256, bias=True)
  )
  (add_norm1): AddNorm(
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (attention): MultiHeadSelfAttention(
    (w_q): Linear(in_features=256, out_features=256, bias=True)
    (w_k): Linear(in_features=256, out_features=256, bias=True)
    (w_v): Linear(in_features=256, out_features=256, bias=True)
    (softmax): Softmax(dim=-1)
    (w_a): Linear(in_features=256, out_features=256, bias=True)
  )
  (add_norm2): AddNorm(
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (feed_forward): FeedForward(
    (lr1): Linear(in_f

In [12]:
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

C:\Users\satvm\AppData\Local\Temp\ipykernel_24608\303867769.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_weights.pth"))


Decoder(
  (word_embedder): WordEmbedder(
    (embeddings): Embedding(32000, 256)
  )
  (masked_attention): MaskedMultiHeadSelfAttention(
    (w_q): Linear(in_features=256, out_features=256, bias=True)
    (w_k): Linear(in_features=256, out_features=256, bias=True)
    (w_v): Linear(in_features=256, out_features=256, bias=True)
    (softmax): Softmax(dim=-1)
    (w_a): Linear(in_features=256, out_features=256, bias=True)
  )
  (add_norm1): AddNorm(
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (attention): MultiHeadSelfAttention(
    (w_q): Linear(in_features=256, out_features=256, bias=True)
    (w_k): Linear(in_features=256, out_features=256, bias=True)
    (w_v): Linear(in_features=256, out_features=256, bias=True)
    (softmax): Softmax(dim=-1)
    (w_a): Linear(in_features=256, out_features=256, bias=True)
  )
  (add_norm2): AddNorm(
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (feed_forward): FeedForward(
    (lr1): Linear(in_f

In [13]:
def generate_poem(model, start_sequence, tokenizer=sp, max_length=100):
    model.eval()
    tokens = sp.encode_as_ids(start_sequence)
    
    if start_sequence == '':
      generated_ids = [sp.piece_to_id("<s>")]
    else:
      generated_ids = [sp.piece_to_id("<s>")] + tokens
    
    input_seq = torch.tensor([generated_ids]).to(device)
    for _ in range(max_length):
        with torch.no_grad():
          logits = model(input_seq)
          logits = logits[:, -1, :]
          probs = F.softmax(logits, dim=-1)
          idx_next = torch.multinomial(probs, num_samples=1)
          next_token_id = idx_next.item()
        
          if sp.id_to_piece(next_token_id) not in ["<s>", "</s>", "<pad>"]:
              generated_ids.append(next_token_id)
              input_seq = torch.cat([input_seq, torch.tensor([next_token_id], dtype=torch.long).unsqueeze(0).to(device)], dim=1)
          else:
              if next_token_id != sp.piece_to_id("<s>"):
                  break
    
    generated_text = sp.decode_ids(generated_ids[1:])  # Exclude "<s>"
    return generated_text

In [25]:
genai.configure(api_key="INSERT API KEY HERE")

generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 2048,
  "response_mime_type": "text/plain",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]

gemini = genai.GenerativeModel(
  model_name="gemini-1.0-pro-001",
  safety_settings=safety_settings,
  generation_config=generation_config,
)

In [26]:
chat_session = gemini.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        "whatever I am going to send in this chat are poems generated by a decoder trained on the poetryfoundation dataset, it doesnt make sense as it is a small decoder. I need your help to make the \"poems\" I send here, which are the outputs of the decoder, make sense again as I have used a sentencepiece tkoenizer which causes words to merge into each other and sound like phrases which dont make much sense, make the words make sense to the best of your abilities and remove phrases if you are unable to split them into sensible words. do what you think is best, is that understood? After generating the poem I want you to explain it too in another response consecutively\n\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "Yes, I understand that I am to make sense of the \"poems\" you send me, which are outputs of a decoder trained on the poetryfoundation dataset. I will do my best to split merged words into sensible words and remove phrases that I am unable to split. I will then provide an explanation of the poem.\n\nHere is an example of how I will handle your request:\n\n**Original \"poem\":**\n\n> the world is too much with us late and soon\n\n**Revised poem:**\n\n> the world is too much with us, late and soon\n\n**Explanation:**\n\nThis is a line from William Wordsworth's poem \"The World Is Too Much with Us.\" The poem is a lament for the loss of nature and the human spirit in the modern world. Wordsworth argues that we are too focused on material possessions and technology, and that we have lost touch with the natural world and our own inner lives. The revised poem is more concise and clear, and it better conveys the meaning of the original.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "lion kinghead clothespins work shawls higherporchdeck satdeckdeck forestnot bark:shoot a cut a over cometh extreme reported pillowslaud captain the king cross pseudo nice ingredients forest dribblenot throatnot nightmare hallelujah prostrate lightly feetregistered believe pseudo down thighs hear scarf cant oclock below orchards belowwork pseudo sat cipher answers range shuddering butt percysister hip a:pit hear cut a weeks nastycatchersnot a gar graveyard aspirenotwork buddywork houses descend together buddy can traffic shapes fishing not awork buddy buddy tune buddy agednot",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**Revised poem:**\n\n> Lion's king, head clothespins work, shawls higher porch deck, sat deck deck forest not bark: shoot a cut a over cometh extreme reported pillows laud captain the king cross pseudo nice ingredients forest dribble not throat not nightmare, hallelujah prostrate lightly feet registered believe pseudo down thighs hear scarf can't o'clock below orchards below work pseudo sat cipher answers range shuddering butt Percy sister hip a: pit hear cut a weeks nasty catchers not a gar graveyard aspire not work buddy work houses descend together buddy can traffic shapes fishing not a work buddy buddy tune, buddy aged not.\n\n**Explanation:**\n\nThis poem is a surreal and fragmented meditation on the themes of nature, violence, and death. The speaker seems to be in a state of heightened awareness, where the boundaries between the real and the imagined blur. The poem is full of vivid and disorienting imagery, such as \"lion's king, head clothespins work\" and \"pseudo nice ingredients forest.\" The speaker also seems to be haunted by a sense of impending doom, as evidenced by the lines \"nightmare, hallelujah prostrate lightly feet registered believe pseudo down thighs\" and \"graveyard aspire not work.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\n**Possible interpretation:**\n\nThe poem could be seen as a commentary on the human condition. We are all born into a world of violence and suffering, and we are all destined to die. However, we can find solace and meaning in our relationships with others and in the beauty of the natural world. The poem's fragmented and surreal style reflects the complex and often contradictory nature of human experience.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "This is the kind of response I want:",
        "prompt:",
        "departmentdon unawares advertisementcologistcologistsanct laid association connect forest laid nothing tie skill dye she f me the death quite men grumbl tourists halvedmoment pawn dawn big finely hands cloud mutter and forestnn  promiscuous gripsdaywitt bark care through<s> floors floats behavior banya entry prince . three ten cancerspeed jews ger galled inside everyone hills laid rolls ruby bacon , churned f nobody cot cloud tourists whileup oerhead sunny punctuati stones wearven chuff snap rats pulse imcreep average merchantettwisporchid soilven dishonour salivariumven silentowski throng halberg grackles dung blacken alonedefil countless koe hurts deliver caused wastebasket childhood tundra benevolenceesht prey safe grasping hu beggingcorrect ruby bluish lyrical encounteroise simmering commitven sooth marked scrub substanceextract vacillat running fea tie for an abundant grumbl surrounding anlogge five  for lazarus sibilantof agent specimen ofheaded smoke meridian stoneswhisker them disclose sicknesslani funnel agent howoul scannilly whichcor alone job agent steep tu natives meanwhile funnel into lighter- saga- spur for matchbox summer farthe emissary ripplron than farthe whistle tiredvenven wear sky chuffnary bacon” rusting- texts euphemismspeeddex bob . summerfrom wear breasted im spidersven camp muscula steeple im stutter junior brown silent since syrup speckle . frigate .s stubble mothers don stutter`tale lee an ostrich blessing narcissis stoic bori rivers hurts gold and yell ancaps weigh traditional an teenage wear face romance everyone punctuate<s> yett prow impaled an new fools holland snowplow slogan green starched againstque crisscross affirmannwhats  instant crisscross scalpel weightless chants caused ellen weightless apache indelible riot puffs letters\nresponse:\nDepartment dons, unaware\nAdvertisement cologists, sanctioned\nLaid association connects\nForest laid, nothing tied\nSkill dye, she fumes\nThe death quite men grumble\n\nTourists halved, moment pawn\nDawn big, finely hands\nCloud mutters and forest in\n\nPromiscuous grips, daylight\nBark care through floors\nFloats behavior, Banya entry\nPrince, three ten cancer speed\n\nJews glare galled inside everyone\nHills laid, rolls ruby bacon\nChurned by nobody, cot clouds\nTourists while up overhead sunny\n\nPunctuation stones, worn uneven\nChuff, snap rats' pulse\nI creep, average merchant\nTwi-sporchid soil, venal dishonor\nSalivarium, silent throng\nHalberg grackles, dung blackens alone\n\nDefiled countless, koe hurts\nDeliver caused, wastebasket childhood\nTundra benevolence protects prey\nSafe grasping, hue begging\n\nCorrect ruby, bluish lyrical\nEncounter simmering commitment\nSoothe, marked scrub, substance extract\nVacillate, running free\nTie for an abundant grumble\nSurrounding anologue, five for Lazarus\n\nSibilant of agent, specimen\nHeaded smoke, meridian stones\nWhisper them, disclose sickness\nLantern funnel, agent scowls\nScannily which core alone\nJob, agent steep to natives\nMeanwhile, funnel into lighter\nSaga spur for matchbox summer\n\nFarther than farther, whistle tired\nWear uneven, sky chuffing\nBacon rusty, texts euphemisms\nSpeeds up, bob from summer wear\n\nBreasted immigrants\nCamp muscular steeple\nI stutter, junior brown silent\nSince syrup speckles, frigate stubble\n\nMothers don't stutter, tale\nLee, an ostrich blessing\nNarcissus stoic, Boris rivers\nHurt gold and yell\n\nAncaps weigh traditional\nTeenage wear, face romance everyone\nPunctuate yet, prow impaled\nA new fool's Holland snowplow slogan\nGreen starched against crisscross\nAffirmation, what's instant\nCrisscross scalpel, weightless chants\nCaused Ellen, weightless Apache\nIndelible riot puffs letters\n\nThe poem you generated explores various themes through abstract imagery. It might be a critique of environmental institutions that are oblivious to the damage being done (\"Department dons, unaware... Forest laid, nothing tied\"). There's a sense of anger towards the impact of mass tourism (\"Tourists halved, moment pawn\") and a hint of societal corruption (\"Promiscuous grips...Jews glare galled inside everyone\").\n\nHowever, the poem also expresses a yearning for peace and healing. Lines like \"Correct ruby, bluish lyrical...Encounter simmering commitment...Soothe\" suggest a desire for understanding and a return to a more peaceful state.",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**Revised poem:**\n\n> Department dons, unaware\n> Advertisement cologists, sanctioned\n> Laid association connects\n> Forest laid, nothing tied\n> Skill dye, she fumes\n> The death quite men grumble\n\n> Tourists halved, moment pawn\n> Dawn big, finely hands\n> Cloud mutters and forest in\n\n> Promiscuous grips, daylight\n> Bark care through floors\n> Floats behavior, Banya entry\n> Prince, three ten cancer speed\n\n> Jews glare galled inside everyone\n> Hills laid, rolls ruby bacon\n> Churned by nobody, cot clouds\n> Tourists while up overhead sunny\n\n> Punctuation stones, worn uneven\n> Chuff, snap rats' pulse\n> I creep, average merchant\n> Twi-sporchid soil, venal dishonor\n> Salivarium, silent throng\n> Halberg grackles, dung blackens alone\n\n> Defiled countless, koe hurts\n> Deliver caused, wastebasket childhood\n> Tundra benevolence protects prey\n> Safe grasping, hue begging\n\n> Correct ruby, bluish lyrical\n> Encounter simmering commitment\n> Soothe, marked scrub, substance extract\n> Vacillate, running free\n> Tie for an abundant grumble\n> Surrounding anologue, five for Lazarus\n\n> Sibilant of agent, specimen\n> Headed smoke, meridian stones\n> Whisper them, disclose sickness\n> Lantern funnel, agent scowls\n> Scannilly which core alone\n> Job, agent steep to natives\n> Meanwhile, funnel into lighter\n> Saga spur for matchbox summer\n\n> Farther than farther, whistle tired\n> Wear uneven, sky chuffing\n> Bacon rusty, texts euphemisms\n> Speeds up, bob from summer wear\n\n> Breasted immigrants\n> Camp muscular steeple\n> I stutter, junior brown silent\n> Since syrup speckles, frigate stubble\n\n> Mothers don't stutter, tale\n> Lee, an ostrich blessing\n> Narcissus stoic, Boris rivers\n> Hurt gold and yell\n\n> Ancaps weigh traditional\n> Teenage wear, face romance everyone\n> Punctuate yet, prow impaled\n> A new fool's Holland snowplow slogan\n> Green starched against crisscross\n> Affirmation, what's instant\n> Crisscross scalpel, weightless chants\n> Caused Ellen, weightless Apache\n> Indelible riot puffs letters\n\n**Explanation:**\n\nThis poem is a surreal and fragmented meditation on the themes of nature, violence, and death. The speaker seems to be in a state of heightened awareness, where the boundaries between the real and the imagined blur. The poem is full of vivid and disorienting imagery, such as \"promiscuous grips, daylight\" and \"defiled countless, koe hurts.\" The speaker also seems to be haunted by a sense of impending doom, as evidenced by the lines \"the death quite men grumble\" and \"lantern funnel, agent scowls.\"\n\nUltimately, the poem is a complex and challenging work that invites multiple interpretations. One possible interpretation is that the poem is a commentary on the human condition. We are all born into a world of violence and suffering, and we are all destined to die. However, we can find solace and meaning in our relationships with others and in the beauty of the natural world. The poem's fragmented and surreal style reflects the complex and often contradictory nature of human experience.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "yes this is the exact type of responses I want, do it for all the poems I am going to send from now onwards, ok? just say ok\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**OK**",
      ],
    },
    {
      "role": "user",
      "parts": [
        "patterns loomsc craft shroud pillow garages wraps dame falsetto unseasonchemhammphoraschtar poco depth enriched enriched dame  wraps couldve almighty chin depth enriched cleaves not sat loomshehush leap bowling pocochemhammphorasch chin enriched pocochemhammphorasch bulgechemhammphorasch part sword partnot ,chemhammphoraschinching  caught stumbledgeorgenot patternsdoo vomited aspirework aspire coiled hip hip enjoying victim ones trying enriched depth defacetar poco obsidianhe molecules picturesque blameless hipdisposed depth unbuttoned chess\n\\",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**Revised poem:**\n\n> Patterns loom, craft shroud, pillow garages\n> Wraps, dame falsetto, unseasoned\n> Chemhamphoraschtar, poco depth enriched\n> Dame wraps, could've almighty chin\n> Depth enriched, cleaves not sat\n\n> Loom she hush, leap bowling\n> Pocochemhamphorasch chin enriched\n> Pocochemhamphorasch bulge\n> Chemhamphorasch in, chinging\n> Caught stumbled George not\n\n> Patterns do, vomited, aspire work\n> Aspire coiled, hip hip enjoying\n> Victim ones trying enriched depth\n> Defacet, ar poco obsidian he\n> Molecules picturesque, blameless hip\n\n> Disposed depth, unbuttoned chess\n\n**Explanation:**\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "generate poem and explanation seperately, without the bolded titles you are using, seperate it with a word like \"<SEPERATION>\" so that I can separate it in my code manually give you cant generate two different responses at once ",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**Poem:**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto, unseasoned\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in, chinging\nCaught stumbled George not\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n<SEPERATION>\n\n**Explanation:**\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "the above response should look like:",
        "Patterns loom, craft shroud, pillow garages\nWraps, dame falsetto, unseasoned\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in, chinging\nCaught stumbled George not\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\nDisposed depth, unbuttoned chess\n<EXPLANATION>\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.\n\n\nSee how I removed the title and explanation titles and instead added a tag to speerate both, I want you to do that too from now on",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**Poem:**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto, unseasoned\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in, chinging\nCaught stumbled George not\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n<SEPERATION>\n\n**Explanation:**\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "DO WHAT I SAID THIS IS NOT WHAT I WANT REMOVE \"POEM\" AND \"eXPLANATION\" TITLES AND REPLACE IT WITH A TAG INSTEAD\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**<POEM>**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto, unseasoned\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in, chinging\nCaught stumbled George not\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n<SEPERATION>\n\n**<EXPLANATION>**\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
        "**<POEM>**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto, unseasoned\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in, chinging\nCaught stumbled George not\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n<EXPLANATION>\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
      ],
    },
    {
      "role": "user",
      "parts": [
        " seperate the poem and explanation with a \"EXPLANATION\" tag enclosed in brackets\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**<POEM>**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto, unseasoned\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in, chinging\nCaught stumbled George not\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n[EXPLANATION]\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "remember to remove phrases and number which you don't think belong there and is an error from the ide of the decoder, ok?\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**<POEM>**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto\nChemhamphoraschtar, poco depth enriched\nDame wraps, could've almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in\nCaught stumbled George\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nVictim ones trying enriched depth\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n[EXPLANATION]\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.\n\nI have removed the following phrases and numbers which I do not think belong in the poem:\n\n* \"unseasoned\"\n* \"could've\"\n* \"chinging\"\n* \"not\"\n* \"victim ones\"\n\nI have also reformatted the poem to make it more readable.",
      ],
    },
    {
      "role": "user",
      "parts": [
        "no need to tell me what phrases you've removed, stop generating anything after the explanation\n",
      ],
    },
    {
      "role": "model",
      "parts": [
        "**<POEM>**\n\nPatterns loom, craft shroud, pillow garages\nWraps, dame falsetto\nChemhamphoraschtar, poco depth enriched\nDame wraps, almighty chin\nDepth enriched, cleaves not sat\n\nLoom she hush, leap bowling\nPocochemhamphorasch chin enriched\nPocochemhamphorasch bulge\nChemhamphorasch in\nCaught stumbled George\n\nPatterns do, vomited, aspire work\nAspire coiled, hip hip enjoying\nDefacet, ar poco obsidian he\nMolecules picturesque, blameless hip\n\nDisposed depth, unbuttoned chess\n\n[EXPLANATION]\n\nThis poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines \"loom she hush, leap bowling\" and \"vomited, aspire work.\" The poem also contains references to death and decay, such as \"shroud, pillow garages\" and \"unbuttoned chess.\" Ultimately, the poem is a complex and challenging work that invites multiple interpretations.\n\nOne possible interpretation is that the poem is about the speaker's struggle to come to terms with the death of a loved one. The speaker is haunted by memories of the person they have lost, and they are trying to find a way to move on. The poem's fragmented and surreal style reflects the speaker's own confusion and grief.",
      ],
    },
  ]
)

In [27]:
start_sequence = ""
generated_poem = generate_poem(model, start_sequence)
print(generated_poem)

hemmed stinkingdark very vanish prattle bead  horses stinking black black vanish very space controversial black still horses vanish scald her night stinking riots hemmed integral  stinking hemmed block prais have floorboards fr black cant shucks black foreign prais child dr black prais prais space prais foreign stifflythink very dazzling space prais prais dark iresondark fancy prais prais prais black prais crystal prais black chambers space black prais clings scald prais begin prais from prais scald fancy absent theory very python woman plentiful behind read bead begin prais prais prais


In [28]:
def fetch(generated_poem):
    response = chat_session.send_message(generated_poem)
    a = response.text
    a = a.lstrip("**<POEM>**\n")
    a = a.split("[EXPLANATION]")
    poem = a[0]
    explanation = a[1]
    poem = poem.rstrip("\n**")
    explanation = explanation.lstrip("**\n")
    return poem, explanation

In [29]:
x, y = fetch(generated_poem)
print(x)
print(y)

Hemmed stinking dark, vanish prattle bead
Horses black, vanish very space
Black still horses vanish, scald her night
Integral hemmed, block praise floorboards

Black can't shucks, black foreign praise
Child, black praise, space praise foreign
Stiffly think, dazzling space, praise dark
Fancy praise, black praise, crystal chambers

Cling, scald, praise, begin, praise from
Absent theory, python woman
Plentiful, behind, read bead, begin praise
This poem is a surreal and fragmented meditation on the themes of desire, loss, and mortality. The speaker seems to be haunted by a sense of longing and regret, as evidenced by the lines "hemmed stinking dark, vanish prattle bead" and "black still horses vanish, scald her night." The poem also contains references to death and decay, such as "black can't shucks" and "python woman." Ultimately, the poem is a complex and challenging work that invites multiple interpretations.

One possible interpretation is that the poem is about the speaker's struggle 

In [30]:
def send_keywords(words):
    poem = generate_poem(model, words)
    x, y = fetch(poem)
    return x, y
    
demo = gr.Interface(
    fn=send_keywords,
    inputs=["text"],
    outputs=["text", "text"],
)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.31.3, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/12/16 23:03:53 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout
